In [1]:
from microphone import DispositivosUSB
from IPython.display import Audio

In [6]:
dispositivos = DispositivosUSB()
ids_microfonos = dispositivos.obtener_ids_microfonos()
print(ids_microfonos)

Micrófono 1: MicrÃ³fono (3- USB Camera), 1 canales
Micrófono 2: MicrÃ³fono (2- USB Camera), 1 canales
Micrófono 3: MicrÃ³fono (USB Camera), 1 canales
Cámara 0 encontrada: 640x480, 30.0 FPS
Cámara 1 encontrada: 640x480, 30.0 FPS
Cámara 2 encontrada: 640x480, 30.0 FPS
No se pudo abrir la cámara 3
Emparejado: Micrófono 1 (USB Camera) con Cámara 0
Emparejado: Micrófono 2 (USB Camera) con Cámara 1
Emparejado: Micrófono 3 (USB Camera) con Cámara 2
[1, 2, 3]


In [26]:
# Camara 1
mic_id = ids_microfonos[0]
if mic_id is not None:
    archivo = dispositivos.grabar_audio(mic_id, 5, "audio_mic1.wav")
    display(Audio(archivo))

Grabando desde micrófono 1 durante 5 segundos...
Grabación completada para micrófono 1


In [27]:
# Camara 3
mic_id = ids_microfonos[2]
if mic_id is not None:
    archivo = dispositivos.grabar_audio(mic_id, 5, "audio_mic3.wav")
    display(Audio(archivo))

Grabando desde micrófono 3 durante 5 segundos...
Grabación completada para micrófono 3


In [5]:
dispositivos.ver_camara_tiempo_real(camara_id=1)

Mostrando video en tiempo real de la cámara 1. Presiona 'q' para salir.


In [17]:
import tensorflow_hub as hub
import tensorflow as tf
import librosa
import numpy as np


In [ ]:
MODEL_PATH = "critical_sound_detector_model.h5"
model = tf.keras.models.load_model(MODEL_PATH)

# Class labels
LABELS = ["Bocina", "Sirena", "null"]


In [13]:
# Load YAMNet model from TensorFlow Hub
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

In [24]:
def predict_audio(audio_path):
    # Cargar el audio
    audio, _ = librosa.load(audio_path, sr=16000)

    # Obtener los embeddings de YAMNet
    _, embeddings, _ = yamnet_model(audio)
    average_embedding = tf.reduce_mean(embeddings, axis=0).numpy()

    # Predecir con el modelo Keras
    predictions = model.predict(np.array([average_embedding]))[0]

    # Emparejar etiquetas con predicciones
    paired_results = list(zip(LABELS, predictions))

    # Ordenar por confianza (de mayor a menor)
    sorted_pairs = sorted(paired_results, key=lambda x: x[1], reverse=True)

    # Mostrar predicciones
    print(f"\n🔊 Prediction for: {audio_path}")
    for label, score in sorted_pairs:
        probability = score * 100
        print(f"{label}: {probability:.2f}%")

In [28]:
predict_audio("audio_mic3.wav")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

🔊 Prediction for: audio_mic3.wav
null: 99.97%
Bocina: 0.02%
Sirena: 0.01%
